In [1]:
import pandas as pd
import os 
import time
import datetime
import functools
from .methods.runner import Runner

In [ ]:
folder = "data"

files = sorted(os.listdir(folder))

runner = Runner(models = None)
min_validation_size = 8
num_steps = 4
validation_split = 0.05
min_size = min_validation_size * (num_steps - 1) / validation_split

for i, file in enumerate(files):
	df = pd.read_csv(os.path.join(folder, file), index_col='Period').sort_index()

	if df.shape[0] < min_size:
		continue

	df = runner.get_df(df)